# National Grid Carbon Intensity API Wrapper

<br>

### Imports

In [1]:
## Data Manipulation
import pandas as pd
import numpy as np

## Plotting
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

## Datetime
from datetime import datetime
import dateutil.parser

## Misc
import requests
import collections
import mysql.connector as mysql

<br>

### Helper Functions

In [2]:
def hide_spines(ax, positions=['top', 'right']):
    """
    Pass a matplotlib axis and list of positions with spines to be removed
    
    args:
        ax:          Matplotlib axis object
        positions:   Python list e.g. ['top', 'bottom']
    """
    assert isinstance(positions, list), 'Position must be passed as a list'
    
    for position in positions:
        ax.spines[position].set_visible(False)
        
def try_accept(exception, default=None):
    def decorator(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            try:
                return func(*args, **kwargs)
            except:
                return exception
        return wrapper
    return decorator

<br>

### Making The First Call

The NG carbon intensity API has a number of endpoints which can be used for different queries. Here we create a function which checks that the end-point is available and then attaches it to the API root url.

In [3]:
def form_stream_url(stream):
    branch_streams = ['intensity', 'regional', 'generation', 'regional/intensity', 'intensity/factors']
    url_root = 'https://api.carbonintensity.org.uk'

    assert stream in branch_streams, f'{branch} is not a recognised API branch, please use one of: {", ".join(branch_streams)}.'

    stream_url = f'{url_root}/{stream}'

    return stream_url

stream = 'intensity'
stream_url = form_stream_url(stream)

stream_url

'https://api.carbonintensity.org.uk/intensity'

<br>

We can make a call to the intensity stream url to see what emissions look like right now

In [4]:
r = requests.get(stream_url)

r.json()

{'data': [{'from': '2019-12-01T17:30Z',
   'to': '2019-12-01T18:00Z',
   'intensity': {'forecast': 253, 'actual': 251, 'index': 'moderate'}}]}

<br>

### Creating Querier for National Emissions & Generation

We start by creating a function which parses datelike strings and objects into the correct format needed for the API

In [5]:
def form_ISO8601_dt(dt): 
    """
    This function takes datetimes in a variety of formats and converts them to a string in the ISO-8601 format.
    If only a date is provided it will set the time to 00:00.
    """
    
    class_type_2_formatter = {
        str : pd.to_datetime,
        pd._libs.tslibs.timestamps.Timestamp : lambda pd_ts: pd_ts,
        datetime.date : lambda datetime_date: datetime.datetime.combine(datetime_date, datetime.datetime.min.time()),
        datetime.datetime : lambda datetime_datetime: datetime_datetime,
    }
    
    assert isinstance(dt, tuple(class_type_2_formatter.keys())), f'The date variable provided could not be understood, please use one of: {", ".join([str(x) for x in list(class_type_2_formatter.keys())])}.'
    
    dt_fmt = class_type_2_formatter[type(dt)](dt)
    dt_ISO = dt_fmt.isoformat() + 'Z'
    
    return dt_ISO

from_date = pd.to_datetime('2019-01-01')
from_date = form_ISO8601_dt(from_date)

from_date

'2019-01-01T00:00:00Z'

<br>

Next we'll create a function which will handles the from and to dates

In [6]:
str_dt_2_pd_ts = lambda str_dt: pd.to_datetime(str_dt[:-1]).tz_localize(None)

def process_n_check_dt_rng(from_date=None, to_date=None, SP_start=True):
    if not from_date:
        from_date = datetime.date.today()
    from_date = form_ISO8601_dt(from_date)
    
    if not to_date:
        to_date = pd.to_datetime(from_date[:-1]) + pd.Timedelta(hours=23, minutes=30)
    to_date = form_ISO8601_dt(to_date)
    
    assert_err_msg = 'The dates provided should be within two weeks of each other'
    assert str_dt_2_pd_ts(to_date[:-1])-str_dt_2_pd_ts(from_date[:-1]) < pd.Timedelta(weeks=2), assert_err_msg
    
    if SP_start == True:
        from_date = form_ISO8601_dt( str_dt_2_pd_ts(from_date[:-1]) + pd.Timedelta(minutes=30) )
        to_date = form_ISO8601_dt( str_dt_2_pd_ts(to_date[:-1]) + pd.Timedelta(minutes=30) )
    
    return from_date, to_date

from_date = '2019-01-01'
to_date = None

from_date, to_date = process_n_check_dt_rng(from_date, to_date)
from_date, to_date

('2019-01-01T00:30:00Z', '2019-01-02T00:00:00Z')

<br>

We can now make the query call and parse the response into a json and then a dataframe

In [7]:
def stream_params_2_json(stream, from_date=None, to_date=None, SP_start=True):
    from_date, to_date = process_n_check_dt_rng(from_date, to_date, SP_start)
    
    stream_url = form_stream_url(stream)
    url = f'{stream_url}/{from_date}/{to_date}'

    r = requests.get(url)
    r_json = r.json()
    
    return r_json

stream = 'intensity'
r_json = stream_params_2_json(stream)

df = pd.DataFrame(r_json['data'])
df.head()

,from,to,intensity
0,2019-12-01T00:00Z,2019-12-01T00:30Z,"{'forecast': 232, 'actual': 229, 'index': 'mod..."
1,2019-12-01T00:30Z,2019-12-01T01:00Z,"{'forecast': 226, 'actual': 230, 'index': 'mod..."
2,2019-12-01T01:00Z,2019-12-01T01:30Z,"{'forecast': 237, 'actual': 228, 'index': 'mod..."
3,2019-12-01T01:30Z,2019-12-01T02:00Z,"{'forecast': 233, 'actual': 227, 'index': 'mod..."
4,2019-12-01T02:00Z,2019-12-01T02:30Z,"{'forecast': 227, 'actual': 226, 'index': 'mod..."


<br>

The main issue we now have is that some of the columns contain multiple values within a single row entry. We can get around this by expanding those columns which are identified as iterable, and by making this functionality recursive we can do full expansions for nested iterables.

In [8]:
def expand_cols(df, cols_2_expand=[]):
    for col in cols_2_expand:
        new_df_cols = df[col].apply(pd.Series)

        df[new_df_cols.columns] = new_df_cols
        df = df.drop(columns=col)

    s_cols_2_expand = df.iloc[0].apply(type).isin([collections.OrderedDict, dict, list, tuple])

    if s_cols_2_expand.sum() > 0:
        cols_2_expand = s_cols_2_expand[s_cols_2_expand].index
        df = expand_cols(df, cols_2_expand)

    return df

def emiss_r_json_2_df(r_json):
    df = pd.DataFrame(r_json['data'])
    df = expand_cols(df)

    df.index = pd.to_datetime(df['from'])
    df.index.name = None

    cols_2_keep = list(set(df.columns) - set(['to', 'index', 'from']))
    df = df[cols_2_keep]

    return df

df = emiss_r_json_2_df(r_json)
df.head()

,forecast,actual
2019-12-01 00:00:00+00:00,232,229.0
2019-12-01 00:30:00+00:00,226,230.0
2019-12-01 01:00:00+00:00,237,228.0
2019-12-01 01:30:00+00:00,233,227.0
2019-12-01 02:00:00+00:00,227,226.0


<br>

We can also use the API to query the percentages of national generation comging from specific fuels

In [9]:
def expand_json_fuel(json_data, data_col='data'):
    fuels_SP = [iterable['generationmix'] for iterable in json_data[data_col]]
    fuels_pct_SP = [(pd.DataFrame(fuels)
                     .set_index('fuel')
                     .iloc[:, 0]
                     .to_dict() 
                    )

                    for fuels in fuels_SP]
    
    return fuels_pct_SP

def gen_r_json_2_df(r_json):
    fuels_pct_SP = expand_json_fuel(r_json)

    df_fuels_pct_SP = pd.DataFrame(fuels_pct_SP)
    df_fuels_pct_SP.index = pd.to_datetime([SP_data['from'] for SP_data in r_json['data']])
    
    return df_fuels_pct_SP

stream = 'generation'

r_json = stream_params_2_json(stream)
df_fuels_pct_SP = gen_r_json_2_df(r_json)

df_fuels_pct_SP.head()

,biomass,coal,imports,gas,nuclear,other,hydro,solar,wind
2019-12-01 00:00:00+00:00,8.6,2.4,10.5,41.3,26.2,0.5,1.2,0.0,9.3
2019-12-01 00:30:00+00:00,8.5,2.4,10.5,41.6,26.2,0.5,1.1,0.0,9.2
2019-12-01 01:00:00+00:00,8.4,2.4,10.6,41.2,26.4,0.5,1.1,0.0,9.4
2019-12-01 01:30:00+00:00,8.7,2.5,10.9,39.2,27.2,0.5,1.2,0.0,9.8
2019-12-01 02:00:00+00:00,8.6,2.5,10.6,39.4,27.1,0.5,1.1,0.0,10.2


<br>

### Creating Querier for Regional Emissions & Generation 

We start by calling the <i>regional/intensity</i> stream which contains both regional fuel percentages and emissions levels

In [10]:
stream = 'regional/intensity'
r_json = stream_params_2_json(stream)

df = pd.DataFrame(r_json['data'])
df.head()

,from,to,regions
0,2019-12-01T00:00Z,2019-12-01T00:30Z,"[{'regionid': 1, 'dnoregion': 'Scottish Hydro ..."
1,2019-12-01T00:30Z,2019-12-01T01:00Z,"[{'regionid': 1, 'dnoregion': 'Scottish Hydro ..."
2,2019-12-01T01:00Z,2019-12-01T01:30Z,"[{'regionid': 1, 'dnoregion': 'Scottish Hydro ..."
3,2019-12-01T01:30Z,2019-12-01T02:00Z,"[{'regionid': 1, 'dnoregion': 'Scottish Hydro ..."
4,2019-12-01T02:00Z,2019-12-01T02:30Z,"[{'regionid': 1, 'dnoregion': 'Scottish Hydro ..."


<br>

The first problem is that each entry in the region column contains two dataframes worth of information relating to  generation and emissions

In [11]:
df_regions_SP = pd.DataFrame(r_json['data'][0]['regions'])

df_regions_SP.head()

,regionid,dnoregion,shortname,intensity,generationmix
0,1,Scottish Hydro Electric Power Distribution,North Scotland,"{'forecast': 222, 'index': 'moderate'}","[{'fuel': 'biomass', 'perc': 0}, {'fuel': 'coa..."
1,2,SP Distribution,South Scotland,"{'forecast': 19, 'index': 'very low'}","[{'fuel': 'biomass', 'perc': 3.6}, {'fuel': 'c..."
2,3,Electricity North West,North West England,"{'forecast': 58, 'index': 'very low'}","[{'fuel': 'biomass', 'perc': 0}, {'fuel': 'coa..."
3,4,NPG North East,North East England,"{'forecast': 68, 'index': 'low'}","[{'fuel': 'biomass', 'perc': 28.8}, {'fuel': '..."
4,5,NPG Yorkshire,Yorkshire,"{'forecast': 258, 'index': 'moderate'}","[{'fuel': 'biomass', 'perc': 46.7}, {'fuel': '..."


<br>

Whereas before we just used <i>expand_cols</i>, in this instance the data is stored in a more convoluted nested structure which can't be as easily expanded. If we do use <i>expand_cols</i> then the fuel entries are repeatedly overwritten leaving us the dataframe below.

In [12]:
expand_cols(df_regions_SP).head()

,regionid,dnoregion,shortname,forecast,index,fuel,perc
0,1,Scottish Hydro Electric Power Distribution,North Scotland,222,moderate,wind,20.9
1,2,SP Distribution,South Scotland,19,very low,wind,6.4
2,3,Electricity North West,North West England,58,very low,wind,8.3
3,4,NPG North East,North East England,68,low,wind,2.0
4,5,NPG Yorkshire,Yorkshire,258,moderate,wind,1.9


<br>

Instead we'll create some helper functions to properly transform the data. We start by initialising the multi-index regional fuel percentage and the normally indexed regional carbon intensity dataframes

In [13]:
fuels = sorted(['biomass', 'coal', 'imports', 'gas', 'nuclear', 'other', 'hydro', 'solar', 'wind'])
regions = list(range(1, 19))

multi_idx_arrs = [sorted(regions*len(fuels)), fuels*len(regions)]
multi_idx = pd.MultiIndex.from_arrays(multi_idx_arrs, names=['region', 'fuel'])

df_region_fuel_pct = pd.DataFrame(columns=multi_idx)
df_region_fuel_pct

Empty DataFrame
Columns: [(1, biomass), (1, coal), (1, gas), (1, hydro), (1, imports), (1, nuclear), (1, other), (1, solar), (1, wind), (2, biomass), (2, coal), (2, gas), (2, hydro), (2, imports), (2, nuclear), (2, other), (2, solar), (2, wind), (3, biomass), (3, coal), (3, gas), (3, hydro), (3, imports), (3, nuclear), (3, other), (3, solar), (3, wind), (4, biomass), (4, coal), (4, gas), (4, hydro), (4, imports), (4, nuclear), (4, other), (4, solar), (4, wind), (5, biomass), (5, coal), (5, gas), (5, hydro), (5, imports), (5, nuclear), (5, other), (5, solar), (5, wind), (6, biomass), (6, coal), (6, gas), (6, hydro), (6, imports), (6, nuclear), (6, other), (6, solar), (6, wind), (7, biomass), (7, coal), (7, gas), (7, hydro), (7, imports), (7, nuclear), (7, other), (7, solar), (7, wind), (8, biomass), (8, coal), (8, gas), (8, hydro), (8, imports), (8, nuclear), (8, other), (8, solar), (8, wind), (9, biomass), (9, coal), (9, gas), (9, hydro), (9, imports), (9, nuclear), (9, other), (9, solar), (9, wind), (10, biomass), (10, coal), (10, gas), (10, hydro), (10, imports), (10, nuclear), (10, other), (10, solar), (10, wind), (11, biomass), (11, coal), (11, gas), (11, hydro), (11, imports), (11, nuclear), (11, other), (11, solar), (11, wind), (12, biomass), ...]
Index: []

[0 rows x 162 columns]

In [14]:
df_region_intensity = pd.DataFrame(columns=regions)
df_region_intensity

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18


<br>

We'll then iterate over the json data and populate the regional generation dataframe

In [15]:
%%time

def reg_SP_2_reg_intensity(regions_SP, regions=list(range(1, 19))): 
    intensity_dicts = [region_SP['intensity'] for region_SP in regions_SP['regions']]
    df_intensity = pd.DataFrame(intensity_dicts, index=regions)
    s_intensity = df_intensity['forecast']
    
    return s_intensity

def reg_SP_2_s_reg_fuel_pct(regions_SP, regions=list(range(1, 19))):
    regions_fuels_pct_SP = expand_json_fuel(regions_SP, data_col='regions')
    df_regions_fuels_pct_SP = pd.DataFrame(regions_fuels_pct_SP, index=regions)

    s_reg_fuel_pct = (df_regions_fuels_pct_SP
                      .unstack()
                      .swaplevel(1, 0)
                      .sort_index()
                     )

    return s_reg_fuel_pct

## Iterating over the data extracting fuel gen and carbon intensity in each region across every SP
for SP_data in r_json['data']:
    start_time = pd.to_datetime(SP_data['from'])
    
    df_region_fuel_pct.loc[start_time] = reg_SP_2_s_reg_fuel_pct(SP_data)
    df_region_intensity.loc[start_time] = reg_SP_2_reg_intensity(SP_data)

Wall time: 2.13 s


<br>

We'll put this all together so that our regional wrapper API follow a similar structure to those at the national level

In [234]:
%%time

def initialise_regional_fuel_df():
    fuels = sorted(['biomass', 'coal', 'imports', 'gas', 'nuclear', 'other', 'hydro', 'solar', 'wind'])
    regions = list(range(1, 19))

    multi_idx_arrs = [sorted(regions*len(fuels)), fuels*len(regions)]
    multi_idx = pd.MultiIndex.from_arrays(multi_idx_arrs, names=['region', 'fuel'])

    df_region_fuel_pct = pd.DataFrame(columns=multi_idx)
    return df_region_fuel_pct

def regional_r_json_2_fuel_emiss_df(r_json):
    ## Creating the regional fuel and intensity dataframes
    df_region_fuel_pct = initialise_regional_fuel_df()
    df_region_intensity = pd.DataFrame(columns=regions)
    
    ## Populating the dataframes
    for SP_data in r_json['data']:
        start_time = pd.to_datetime(SP_data['from'])

        df_region_fuel_pct.loc[start_time] = reg_SP_2_s_reg_fuel_pct(SP_data)
        df_region_intensity.loc[start_time] = reg_SP_2_reg_intensity(SP_data)
        
    return df_region_fuel_pct, df_region_intensity
    
def regional_r_json_2_fuel_df(r_json):
    ## Creating the dataframe
    df_region_intensity = pd.DataFrame(columns=regions)
    
    ## Populating the dataframe
    for SP_data in r_json['data']:
        start_time = pd.to_datetime(SP_data['from'])
        df_region_fuel_pct.loc[start_time] = reg_SP_2_s_reg_fuel_pct(SP_data)
        
    return df_region_fuel_pct
    
def regional_r_json_2_emiss_df(r_json):
    ## Creating the dataframe
    df_region_fuel_pct = initialise_regional_fuel_df()
    
    ## Populating the dataframe
    for SP_data in r_json['data']:
        start_time = pd.to_datetime(SP_data['from'])
        df_region_intensity.loc[start_time] = reg_SP_2_reg_intensity(SP_data)
        
    return df_region_intensity

stream = 'regional/intensity'
from_date = '2019-03-01'
to_date = '2019-03-02'

r_json = stream_params_2_json(stream, from_date, to_date)
df_region_fuel_pct, df_region_intensity = regional_r_json_2_fuel_emiss_df(r_json)

Wall time: 4.36 s


In [18]:
df_region_fuel_pct.head(2)

region                         1                                          \
fuel                      biomass coal   gas hydro imports nuclear other   
2019-03-01 00:00:00+00:00     0.0  0.0  61.2  24.9     0.0     0.0   0.0   
2019-03-01 00:30:00+00:00     0.0  0.0  63.8  23.6     0.0     0.0   0.0   

region                                     2   ...    17      18             \
fuel                      solar  wind biomass  ...  wind biomass coal   gas   
2019-03-01 00:00:00+00:00   0.0  13.9     4.3  ...  16.6     7.6  0.0  49.1   
2019-03-01 00:30:00+00:00   0.0  12.6     4.7  ...  16.5     7.6  0.0  50.0   

region                                                            
fuel                      hydro imports nuclear other solar wind  
2019-03-01 00:00:00+00:00   1.3    10.8    23.0   0.3   0.0  7.9  
2019-03-01 00:30:00+00:00   1.2    10.7    22.7   0.2   0.0  7.6  

[2 rows x 162 columns]

In [19]:
df_region_intensity.tail(2)

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
2019-03-01 00:00:00+00:00,243,57,106,16,272,228,365,291,376,149,124,296,245,245,233,116,315,239
2019-03-01 00:30:00+00:00,250,46,112,18,276,222,367,285,377,148,91,315,254,246,232,122,318,243


<br>

We can now easily query the forecast and observed national carbon intensity

In [22]:
stream = 'intensity'

r_json = stream_params_2_json(stream, from_date, to_date)
df_emissions = emiss_r_json_2_df(r_json)

df_emissions.head(2)

,forecast,actual
2019-03-01 00:00:00+00:00,241,239
2019-03-01 00:30:00+00:00,235,243


<br>

As well as national fuel mix 

In [23]:
stream = 'generation'

r_json = stream_params_2_json(stream, from_date, to_date)
df_fuels_pct = gen_r_json_2_df(r_json)

df_fuels_pct.head(2)

,biomass,coal,imports,gas,nuclear,other,hydro,solar,wind
2019-03-01 00:00:00+00:00,7.6,0.0,10.8,49.1,23.0,0.3,1.3,0.0,7.9
2019-03-01 00:30:00+00:00,7.6,0.0,10.7,50.0,22.7,0.2,1.2,0.0,7.6


<br>

And regional carbon intensity

In [236]:
stream = 'regional/intensity'

r_json = stream_params_2_json(stream, from_date, to_date)
df_region_intensity = regional_r_json_2_emiss_df(r_json)

df_region_intensity.head(2)

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
2019-03-01 00:00:00+00:00,243,57,106,16,272,228,365,291,376,149,124,296,245,245,233,116,315,239
2019-03-01 00:30:00+00:00,250,46,112,18,276,222,367,285,377,148,91,315,254,246,232,122,318,243


<br>

And regional fuel generation

In [235]:
stream = 'regional/intensity'

r_json = stream_params_2_json(stream, from_date, to_date)
df_region_fuel_pct = regional_r_json_2_fuel_df(r_json)

df_region_fuel_pct.head(2)

region                         1                                          \
fuel                      biomass coal   gas hydro imports nuclear other   
2019-03-01 00:00:00+00:00     0.0  0.0  61.2  24.9     0.0     0.0   0.0   
2019-03-01 00:30:00+00:00     0.0  0.0  63.8  23.6     0.0     0.0   0.0   

region                                     2   ...    17      18             \
fuel                      solar  wind biomass  ...  wind biomass coal   gas   
2019-03-01 00:00:00+00:00   0.0  13.9     4.3  ...  16.6     7.6  0.0  49.1   
2019-03-01 00:30:00+00:00   0.0  12.6     4.7  ...  16.5     7.6  0.0  50.0   

region                                                            
fuel                      hydro imports nuclear other solar wind  
2019-03-01 00:00:00+00:00   1.3    10.8    23.0   0.3   0.0  7.9  
2019-03-01 00:30:00+00:00   1.2    10.7    22.7   0.2   0.0  7.6  

[2 rows x 162 columns]

<br>

### Creating the API Wrapper Class

To make the API wrapper seem more intuitive we'll separate the regional emissions and generation data

``` python
NG_CI.query_API(from_date, to_date, level='national', data_stream='emissions')
```

In [ ]:
## If a region id or postcode is specified then it should be able to use 
## the national level helper functions for parsing the data

## Should also change regional emissions to be similar to fuels and make it include both the forecast and actual value
## Ideally should aim to generalise so one function is applicable to both

## Perhaps this shouldn't be a class but the flow should be highly logical and visualised

## Should remove ability to query regional emissions and fuel in one go
## part of the trade-off for 80:20 and would improve the ETL clarity

<br>

### Orchestrating Multiple Queries

First we create a list of date danges which we can later iteratively call over

In [62]:
start_date = '2017-01-01'
end_date = '2019-12-01'

dt_rng_open = pd.date_range(start_date, end_date, freq='14D')
dt_rng_close = dt_rng_open + pd.Timedelta(days=14, minutes=-30)

for dt_rng in [dt_rng_open, dt_rng_close]:
    dt_rng.freq = None

dt_rng_pairs = list(zip(dt_rng_open, dt_rng_close))
print(f'Date Range: {dt_rng_pairs[0]} ... {dt_rng_pairs[-1]}')

Date Range: (Timestamp('2017-01-01 00:00:00'), Timestamp('2017-01-14 23:30:00')) ... (Timestamp('2019-12-01 00:00:00'), Timestamp('2019-12-14 23:30:00'))


<br>

Then we carry out the API queries

In [63]:
%%time

stream = 'intensity'
df_emissions = pd.DataFrame()

for dt_rng_pair in dt_rng_pairs:
    try:
        from_date, to_date = dt_rng_pair
        r_json = stream_params_2_json(stream, from_date, to_date)

        df_dt_rng_emissions = emiss_r_json_2_df(r_json)
        df_emissions = df_emissions.append(df_dt_rng_emissions)
    
    except:
        pass
    
df_emissions.head()

Wall time: 41.2 s


,forecast,actual
2017-09-11 23:00:00+00:00,134,140.0
2017-09-11 23:30:00+00:00,143,144.0
2017-09-12 00:00:00+00:00,137,142.0
2017-09-12 00:30:00+00:00,134,140.0
2017-09-12 01:00:00+00:00,133,139.0


<br>

For obvious reasons we don't have any observed values for datetimes which are in the future

In [68]:
df_emissions.tail()

,forecast,actual
2019-12-06 15:30:00+00:00,177,NaN
2019-12-06 16:00:00+00:00,177,NaN
2019-12-06 16:30:00+00:00,177,NaN
2019-12-06 17:00:00+00:00,179,NaN
2019-12-06 17:30:00+00:00,182,NaN


<br>

There are a few other dates which don't have any observed data but are historic. As we do have the forecast values we know this is most likely an issue with the carbon intensity API rather than the wrapper we've created

In [233]:
df_emissions = df_emissions.asfreq('30T')

df_emissions[:end_date].isnull().sum()

forecast      0
actual      337
dtype: int64

<br>

Perhaps the most important part, we'll now save the data

In [270]:
(df_emissions
 .astype(pd.Int32Dtype())
 .reset_index()
 .rename(columns={'index':'datetime'})
 .to_csv('data/emissions.csv', index=False)
)